In [7]:
import json
import numpy as np
from scipy.spatial.transform import Rotation as R

with open("../drawing/drawing_config.json", "r") as f:
    config = json.load(f)

T, u, v, normal, origin = np.array(config["T"]), np.array(config["u"]), np.array(config["v"]), np.array(config["normal"]), np.array(config["origin"])

poses = config["poses"]
print("origin: ", origin)
print("u: ", u)
print("v: ", v)
print("normal: ", normal)
print("")


origin:  [ 0.23817554  0.01953333 -0.02409203]
u:  [ 0.35047046 -0.93515956  0.05144949]
v:  [0.93474657 0.35268831 0.04312556]
normal:  [-0.05847492  0.032978    0.99774402]



In [25]:
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

def arrow(start, vec, color, name=""):
    return go.Scatter3d(
        x=[start[0], start[0] + vec[0]],
        y=[start[1], start[1] + vec[1]],
        z=[start[2], start[2] + vec[2]],
        mode="lines+markers",
        line=dict(color=color, width=4),
        marker=dict(size=3, color=color),
        name=name
    )

def plot_poses(T, u,v,origin, normal, poses, fig):



    # pose positions
    for i, s in enumerate(poses):
        pos = np.array([s["ee.x"], s["ee.y"], s["ee.z"]])
        rotvec = np.array([s["ee.wx"], s["ee.wy"], s["ee.wz"]])

        # --- Convert to rotation matrix ---
        rot = R.from_rotvec(rotvec)
        R_mat = rot.as_matrix()

        # --- Original coordinate axes ---
        x_axis = np.array([1, 0, 0])
        y_axis = np.array([0, 1, 0])
        z_axis = np.array([0, 0, 1])

        # --- Rotated axes ---
        x_rot = R_mat @ x_axis
        y_rot = R_mat @ y_axis
        z_rot = R_mat @ z_axis
        fig.add_trace(arrow(pos, x_rot * 0.01, 'red'))
        fig.add_trace(arrow(pos, y_rot * 0.01, 'blue'))
        fig.add_trace(arrow(pos, z_rot * 0.01, 'green'))



fig = go.Figure()

# Axes
fig.add_trace(arrow(origin, u * 0.01, "red", "u"))
fig.add_trace(arrow(origin, v * 0.01, "green", "v"))
fig.add_trace(arrow(origin, normal * 0.01, "black", "normal"))

plot_poses(T, u, v, origin, normal, poses, fig)
    # Layout
r = 0.1
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[origin[0] - r, origin[0] + r], title='X'),
        yaxis=dict(range=[origin[1] - r, origin[1] + r], title='Y'),
        zaxis=dict(range=[origin[2] - r, origin[2] + r], title='Z'),
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    title="Interactive TCP Visualization",
)
fig.show()
print("num poses: ", len(poses))

num poses:  8


In [32]:
#1 Flip normal direction
normal_flipped = -normal
fig = go.Figure()
fig.add_trace(arrow(origin, normal_flipped * 0.01, 'black'))

#2 Derive new u vector from average pose x axes
x_axes = []
for s in poses:
    rotvec = np.array([s["ee.wx"], s["ee.wy"], s["ee.wz"]])
    rot = R.from_rotvec(rotvec)
    R_mat = rot.as_matrix()
    x_axis = R_mat @ np.array([1, 0, 0])
    x_axes.append(x_axis)
x_axes = np.array(x_axes)
u_new = np.mean(x_axes, axis=0)
u_new /= np.linalg.norm(u_new)

#3 Make sure u_new is orthogonal to normal_flipped
v_new = np.cross(normal_flipped, u_new)
u_new = np.cross(v_new, normal_flipped)
u_new /= np.linalg.norm(u_new)
v_new /= np.linalg.norm(v_new)
fig.add_trace(arrow(origin, u_new * 0.01, 'red'))

# check handedness / transform axis to transformation matrix
R_new = np.column_stack((u_new, v_new, normal_flipped))
print("Determinant of R_new (should be 1): ", np.linalg.det(R_new))

# get vec rot
rot_new = R.from_matrix(R_new).as_rotvec()
print("New rotation vector (radians): ", rot_new)


#4 Define y axis as cross product
v_new = np.cross(normal_flipped, u_new)
v_new /= np.linalg.norm(v_new)
fig.add_trace(arrow(origin, v_new * 0.01, 'green'))

    # Layout

plot_poses(T, u, v, origin, normal, poses, fig)
r = 0.1
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[origin[0] - r, origin[0] + r], title='X'),
        yaxis=dict(range=[origin[1] - r, origin[1] + r], title='Y'),
        zaxis=dict(range=[origin[2] - r, origin[2] + r], title='Z'),
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    title="Interactive TCP Visualization",
)
fig.show()

Determinant of R_new (should be 1):  0.9999999999999999
New rotation vector (radians):  [-0.17407946  3.07960945 -0.05593242]


In [37]:
# transform rot vec to rot matrix and back to verify
R_verify = R.from_rotvec(rot_new).as_matrix()
print("Difference between R_new and R_verify (should be close to 0): ", np.linalg.norm(R_new - R_verify))
#convert rot mat to axis vectors
u_verify = R_verify[:, 0]
v_verify = R_verify[:, 1]
normal_verify = R_verify[:, 2]
print("Difference between u_new and u_verify (should be close to 0): ", np.linalg.norm(u_new - u_verify))
print("Difference between v_new and v_verify (should be close to 0): ", np.linalg.norm(v_new - v_verify))
print("Difference between normal_flipped and normal_verify (should be close to 0): ", np.linalg.norm(normal_flipped - normal_verify))

Difference between R_new and R_verify (should be close to 0):  8.744658280452999e-16
Difference between u_new and u_verify (should be close to 0):  5.93995257398358e-16
Difference between v_new and v_verify (should be close to 0):  2.3107662735148295e-16
Difference between normal_flipped and normal_verify (should be close to 0):  6.005250886403581e-16


In [45]:

# create 2D circle path in meters, center (0,0) radius 0.05
angles = np.linspace(0, 2 * np.pi, 30)
circle2d = [(0.05 * np.cos(a), 0.05 * np.sin(a)) for a in angles]

# convert to world poses and execute
hover_h = 0.04  # 2 cm above plane
contact_z = 0.0  # exactly on plane, or small negative for slight pressure

fig = go.Figure()
for x2d, y2d in circle2d:
    # point in plane coordinates
    point_plane = origin + u_verify * x2d + v_verify * y2d
    fig.add_trace(arrow(point_plane, normal_verify * 0.01, 'black'))
    fig.add_trace(arrow(point_plane, v_verify * 0.01, 'green'))
    fig.add_trace(arrow(point_plane, u_verify * 0.01, 'red'))

for x2d, y2d in circle2d:
    # point in plane coordinates
    point_plane = origin + u_verify * x2d + v_verify * y2d
    point_plane -= normal_verify * hover_h  # hover above plane
    fig.add_trace(arrow(point_plane, normal_verify * 0.01, 'grey'))
    fig.add_trace(arrow(point_plane, v_verify * 0.01, 'aquamarine'))
    fig.add_trace(arrow(point_plane, u_verify * 0.01, 'crimson'))

r = 0.1
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[origin[0] - r, origin[0] + r], title='X'),
        yaxis=dict(range=[origin[1] - r, origin[1] + r], title='Y'),
        zaxis=dict(range=[origin[2] - r, origin[2] + r], title='Z'),
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    title="Interactive TCP Visualization",
)
fig.show()


In [48]:
def fit_drawing_rotation(poses):
    from scipy.spatial.transform import Rotation as R

    rotvecs = np.array([[p["ee.wx"], p["ee.wy"], p["ee.wz"]] for p in poses])
    R_mats = R.from_rotvec(rotvecs).as_matrix()

    # Average rotation matrix
    R_avg = np.mean(R_mats, axis=0)

    # Re-orthogonalize using SVD
    U, _, Vt = np.linalg.svd(R_avg)
    R_ortho = U @ Vt

    rotvec_avg = R.from_matrix(R_ortho).as_rotvec()
    return rotvec_avg

rotvec_fitted = fit_drawing_rotation(poses)
#compare to previous
print("Fitted rotation vector (radians): ", rotvec_fitted)
rot_mat = R.from_rotvec(rotvec_fitted).as_matrix()
u_fitted = rot_mat[:, 0]
v_fitted = rot_mat[:, 1]
normal_fitted = rot_mat[:, 2]
fig = go.Figure()
fig.add_trace(arrow(origin, u_fitted * 0.01, 'red'))
fig.add_trace(arrow(origin, v_fitted * 0.01, 'green'))
fig.add_trace(arrow(origin, normal_fitted * 0.01, 'black'))
r = 0.1
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[origin[0] - r, origin[0] + r], title='X'),
        yaxis=dict(range=[origin[1] - r, origin[1] + r], title='Y'),
        zaxis=dict(range=[origin[2] - r, origin[2] + r], title='Z'),
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    title="Interactive TCP Visualization",
)
fig.show()



Fitted rotation vector (radians):  [-0.17465193  2.63074994  0.04360058]
